In [1]:
import pandas as pd
from pandas_datareader import data
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
tickers = ['CERN','ETFC','FAST','GRMN','ILMN','ROST',
          'VRTX','WLTW','CHTR','CTXS','NFLX']
tickers

['CERN',
 'ETFC',
 'FAST',
 'GRMN',
 'ILMN',
 'ROST',
 'VRTX',
 'WLTW',
 'CHTR',
 'CTXS',
 'NFLX']